<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://gitlab.com/AkashNarayanan" target="_blank"> AkashNarayanan B</a></td>    
</table>
<hr>

Let us look at the different ways of converting a problem between QUBO, Ising and BQM formulations.

# Constructing a BQM from a QUBO Model

## QUBO - NumPy Representation

We have already learned how to define a QUBO matrix as a NumPy matrix. It is possible to construct a Binary Quadratic Model from a NumPy matrix using the `from_numpy_matrix` method.

Let's consider the following objective function

$$f(x_1, x_2, x_3, x_4) = - 5x_1 - 3x_2 - 8x_3 - 6x_4 + 4x_1 x_2 + 8x_1 x_3 + 2x_2 x_3 + 10x_3 x_4$$

The QUBO matrix Q for the objective function is 

$$
Q = \begin{bmatrix}
        -5  &  4   &  8   &  0  \\ 
        0   &  -3  &  2   &  0  \\ 
        0   &  0   &  -8  &  10  \\ 
        0   &  0   &  0   &  -6  \\ 
    \end{bmatrix}
$$

The NumPy matrix `Q` is

In [1]:
import numpy as np

Q = np.array([[ -5,  4,  8,  0],
              [  0, -3,  2,  0],
              [  0,  0, -8, 10],
              [  0,  0,  0, -6]])

## `from_numpy_matrix` Method

### Parameters

- `Q` - The QUBO as a NumPy matrix
- `offset` (optional) - Constant offset

Now let's create a BQM from the above NumPy matrix. We have to pass `Q` as an argument to the `from_numpy_matrix()` method of the `BinaryQuadraticModel` class.

In [2]:
import dimod

bqm_np = dimod.BinaryQuadraticModel.from_numpy_matrix(Q)

print(bqm_np)

BinaryQuadraticModel({0: -5.0, 1: -3.0, 2: -8.0, 3: -6.0}, {(1, 0): 4.0, (2, 0): 8.0, (2, 1): 2.0, (3, 2): 10.0}, 0.0, 'BINARY')


/tmp/ipykernel_26519/671378130.py:3: DeprecationWarning: BQM.from_numpy_matrix(M) is deprecated since dimod 0.10.0, use BQM(M, "BINARY") instead.
  bqm_np = dimod.BinaryQuadraticModel.from_numpy_matrix(Q)


So far we have used strings as keys to represent the variables in linear and quadratic arguments of a BQM class. It is also valid to use integers as keys. In the above output, the integers represent the position of the values in the matrix.

For example, the first term of the linear part `0: -5.0` represents the value `-5.0` at the position `(0, 0)` in the matrix. The first term of the quadratic part `(0, 1): 4` represents the value `4` at the position `(0, 1)` in the matrix.

<div class="alert alert-block alert-info">
The keys for the linear and quadratic arguments of the BQM class can either be strings or integers.
</div>

## Task 1

Create a QUBO in NumPy matrix form for the following objective function and create a BQM from it.

$$f(x_1, x_2, x_3, x_4) = 3x_1 - 7x_2 + 11x_3 - x_4 + 9x_1 x_2 + x_1 x_3 + 2x_2 x_3 + 8x_3 x_4$$

[Click Here for Solution](BQM_Conversion_between_Formulations.ipynb#Task-1)

## QUBO - Dictionary Representation

We can also represent a QUBO problem as a dictionary. What is the need for it you may ask? Dictionary representation can be very helpful for problems with a large number of variables.

In the dictionary representation, only the non-zero terms of a QUBO matrix are considered. This saves up space and improves the efficiency of the problem solving process.

Let's consider a $3 \times 3$ matrix

$$
Q = \begin{bmatrix}
        Q_{11} & Q_{12} & Q_{13}  \\ 
        0      & Q_{22} & Q_{23}  \\ 
        0      & 0      & Q_{33}  \\ 
    \end{bmatrix}
$$

In the dictionary representation, the keys should be the binary variables and their values should be the coefficients associated with these binary variables. The variables can be represented either as a tuple of variable names or as a tuple of numbers. The key for the term $Q_{11}$ in the above matrix can be represented as

- `('x1', 'x1')` - Tuple of variable names
- `(0, 0)` - Tuple of numbers that indicate the position of the term in the matrix

$\color{red}{\text{
The advantage of dictionary representation becomes apparent when we consider a large QUBO matrix.}}$

<div class="alert alert-block alert-danger">
To be discussed in meeting.

This example has to be improved.
</div>

$$
Q_L = \begin{bmatrix}
        \bf{3} & 0 & 0 & 0 & 0 & \bf{4} & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & \bf{9} & 0 \\
        0 & 0 & 0 & \bf{1} & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & \bf{4} & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & \bf{8} \\
      \end{bmatrix}
$$

$\color{red}{\text{The above matrix $Q_L$ can be represented as}}$

In [3]:
Q_Large = {('x1', 'x1'): 3, ('x4', 'x4'): 1, ('x5', 'x5'): 4,
           ('x8', 'x8'): 8, ('x1', 'x6'): 4, ('x3', 'x7'): 9}

$\color{red}{\text{How simple and concise was that?!}}$

A BQM can be constructed from a QUBO problem in dictionary form using the `from_qubo` method of the `BinaryQuadraticModel` class.

## `from_qubo` Method

### Parameters

- `Q` - The QUBO in dictionary form
- `offset` (optional) - Constant offset

Now let's a create a BQM by passing the dictionary `Q_Large` as an argument to the `from_qubo` method.

In [4]:
bqm_qubo = dimod.BinaryQuadraticModel.from_qubo(Q_Large)

print(bqm_qubo)

BinaryQuadraticModel({'x1': 3.0, 'x4': 1.0, 'x5': 4.0, 'x8': 8.0, 'x6': 0.0, 'x3': 0.0, 'x7': 0.0}, {('x6', 'x1'): 4.0, ('x7', 'x3'): 9.0}, 0.0, 'BINARY')


## Task 2

Create a QUBO in dictionary form for the following objective function and create a BQM from it.

$$f(x_1, x_2, x_3, x_4) = 3x_1 - 7x_2 + 11x_3 - x_4 + 9x_1 x_2 + x_1 x_3 + 2x_2 x_3 + 8x_3 x_4$$

[Click Here for Solution](BQM_Conversion_between_Formulations.ipynb#Task-2)

# Constructing a QUBO Model from a BQM

It is also possible to construct a QUBO NumPy matrix from a Binary Quadratic Model.

## `to_numpy_matrix` Method

### Parameter

- `variable_order` - Variable order as a list should be passed as an argument if there are variable names(strings) as keys in the BQM.

Let us consider the following BQM where the keys for the `linear` and `quadratic` arguments are variable names as strings.

In [5]:
bqm_str = dimod.BinaryQuadraticModel(
    {"x1": -5.0, "x2": -3.0, "x3": -8.0, "x4": -6.0},
    {("x1", "x2"): 4, ("x1", "x3"): 8, ("x2", "x3"): 2, ("x3", "x4"): 10},
    0,
    "BINARY",
)

In this case, the variable order as a list should be passed as an argument to the `to_numpy_matrix` method. Without that there would be an error.

In [6]:
np_mat_str = bqm_str.to_numpy_matrix(["x1", "x2", "x3", "x4"])

print(np_mat_str)

[[-5.  4.  8.  0.]
 [ 0. -3.  2.  0.]
 [ 0.  0. -8. 10.]
 [ 0.  0.  0. -6.]]


/tmp/ipykernel_26519/507079686.py:1: DeprecationWarning: bqm.to_numpy_matrix() is deprecated since dimod 0.10.0
  np_mat_str = bqm_str.to_numpy_matrix(["x1", "x2", "x3", "x4"])


If the keys of the `linear` and `quadratic` arguments are numbers, the `variable_order` is optional.

In [7]:
bqm_num = dimod.BinaryQuadraticModel(
    {0: -5.0, 1: -3.0, 2: -8.0, 3: -6.0},
    {(0, 1): 4, (0, 2): 8, (1, 2): 2, (2, 3): 10},
    0.0,
    "BINARY",
)

np_mat_num = bqm_num.to_numpy_matrix()

print(np_mat_num)

[[-5.  4.  8.  0.]
 [ 0. -3.  2.  0.]
 [ 0.  0. -8. 10.]
 [ 0.  0.  0. -6.]]


/tmp/ipykernel_26519/202927046.py:8: DeprecationWarning: bqm.to_numpy_matrix() is deprecated since dimod 0.10.0
  np_mat_num = bqm_num.to_numpy_matrix()


## `to_qubo` Method

This method can be used to construct a QUBO model from a Binary Quadratic Model. If the `vartype` of the BQM is `'SPIN'`, it is converted to `'BINARY'`.

This method returns a tuple of form `(biases, offset)` where `biases` is a dictionary of the linear and quadratic terms and `offset` is a number.

Let's consider the same `bqm_str` used in the previous example. The QUBO form of the BQM is

In [8]:
qubo_str = bqm_str.to_qubo()

print(qubo_str)

({('x2', 'x1'): 4.0, ('x3', 'x1'): 8.0, ('x3', 'x2'): 2.0, ('x4', 'x3'): 10.0, ('x1', 'x1'): -5.0, ('x2', 'x2'): -3.0, ('x3', 'x3'): -8.0, ('x4', 'x4'): -6.0}, 0.0)


In the above output,

- The first terms of the tuple correspond to the linear and quadratic terms of the QUBO
    
    ```python
    {('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10,
     ('x1', 'x1'): -5.0, ('x2', 'x2'): -3.0, ('x3', 'x3'): -8.0, ('x4', 'x4'): -6.0}
    ```
    
- The second term corresponds to the offset `0`.    

# Constructing a BQM from an Ising Model

## Ising Model - Matrix Representation

Unlike the QUBO form, an Ising Model cannot be completely represented by a single matrix. Let's quickly recall the objective function of an Ising Model

$$E(s) = \sum\limits_{i=1}^N h_i s_i + \sum\limits_{i<j}^N J_{i,j} s_i s_j   \qquad\qquad s_i, s_j \in\{-1,+1\}$$

where,

- $s_i, s_j$ - The binary variables correspond to physical Ising spins
- $h_i$ - The linear terms are the biases
- $J_{i,j}$ - The quadratic terms are the couplings between spins

We can completely represent an Ising problem using a vector and a matrix.

- The linear terms $h_i$ should be defined as a vector
- The quadratic terms $J_{i,j}$ should be defined as an upper-triangular matrix

Let us see how this is done using an example. Consider the following objective function

$$f(s_1, s_2, s_3, s_4) = 3s_1 + s_2 + 4s_3 + 2s_4 + s_1 s_3 + 6s_1 s_4 + 7s_3 s_4 + 4s_1 s_2$$

- The linear terms are $3s_1 + s_2 + 4s_3 + 2s_4$. They can be represented as the following vector

$$
h = \begin{bmatrix}
        3  &  1   &  4   &  2  \\ 
    \end{bmatrix}
$$

- The quadratic terms are $s_1 s_3 + 6s_1 s_4 + 7s_3 s_4 + 4s_1 s_2$. They can be represented as the following matrix

$$
J = \begin{bmatrix}
        0  &  4   &  1   &  6  \\ 
        0   &  0  &  0   &  0  \\ 
        0   &  0   &  0  &  7  \\ 
        0   &  0   &  0  & 0  \\ 
    \end{bmatrix}
$$

## Ising Model - Dictionary Representation

As we have already discussed, it is good to know how to represent our problem as a dictionary. We can define the vector $h$ and the matrix $J$ as two separate dictionaries.

The keys of the dictionary can either be variable names or numbers indicating the position of a particular term.

The dictionary representation of vector $h$ is

In [9]:
h = {'s1': 3, 's2': 1,
     's3': 4, 's4': 2}

The dictionary representation of the matrix $J$ is

In [10]:
J = {('s1', 's2'): 4, ('s1', 's3'): 1,
     ('s1', 's4'): 6, ('s3', 's4'): 7}

A Binary Quadratic Model can be constructed from an Ising Model using the `from_ising` method of the `BinaryQuadraticModel` class.

## `from_ising` Method

### Parameters

- `h` - The linear terms should be passed as a dictionary or a list. If it is passed as a list, the indices would be the variable labels.
- `J` - The quadratic terms should be passed as a dictionary.
- `offset` (optional) - Constant offset

Let us create a BQM from the objective function used in the previous example. The linear and quadratic terms are already defined as `h` and `J`. We can simply pass these as arguments to the `from_ising` method.

In [11]:
import dimod

h = {'s1': 3, 's2': 1,
     's3': 4, 's4': 2}

J = {('s1', 's2'): 4, ('s1', 's3'): 1,
     ('s1', 's4'): 6, ('s3', 's4'): 7}

bqm_ising = dimod.BinaryQuadraticModel.from_ising(h, J)

print(bqm_ising)

BinaryQuadraticModel({'s1': 3.0, 's2': 1.0, 's3': 4.0, 's4': 2.0}, {('s2', 's1'): 4.0, ('s3', 's1'): 1.0, ('s4', 's1'): 6.0, ('s4', 's3'): 7.0}, 0.0, 'SPIN')


# Constructing an Ising Model from a BQM

Similarly an Ising Model can be constructed from a Binary Quadratic Model using the `to_ising` method of the `BinaryQuadraticModel` class. If the `vartype` of the BQM is `'BINARY'`, it is converted to `'SPIN'`.

## `to_ising` Method

This method returns a tuple of form `(linear, quadratic, offset)` where `linear` and `quadratic` are dictionaries and `offset` is a number.

Let's consider the following BQM instance

In [12]:
bqm = dimod.BinaryQuadraticModel({'s1': 3.0, 's2': 1.0, 's3': 4.0, 's4': 2.0},
                                 {('s1', 's2'): 4, ('s1', 's3'): 1,
                                  ('s1', 's4'): 6, ('s3', 's4'): 7}, 
                                 0, 
                                 'SPIN')

Now let's convert it to the Ising form using the `to_ising` method.

In [13]:
ising = bqm.to_ising()

print(ising)

({'s1': 3.0, 's2': 1.0, 's3': 4.0, 's4': 2.0}, {('s2', 's1'): 4.0, ('s3', 's1'): 1.0, ('s4', 's1'): 6.0, ('s4', 's3'): 7.0}, 0.0)


In the above output,

- The first term of the tuple corresponds to the linear terms
    ```python
    {'s1': 3.0, 's2': 1.0, 's3': 4.0, 's4': 2.0}
    ```
- The second term of the tuple corresponds to the quadratic terms

    ```python
    {('s1', 's2'): 4, ('s1', 's3'): 1, ('s1', 's4'): 6, ('s3', 's4'): 7}
    ```
   
- The third term corresponds to the offset `0`.    